In [1]:
import re
import logging
import numpy as np
import pandas as pd
import multiprocessing

from re import sub
from time import time 
from unidecode import unidecode
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser

import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random
import numpy as np
import pandas as pd
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
import glob
from gensim.models.phrases import Phrases, Phraser

In [2]:
path_test_neg=r'C:\Users\raja4\Downloads\aclImdb_v1.tar\aclImdb\test\neg'
path_test_pos=r'C:\Users\raja4\Downloads\aclImdb_v1.tar\aclImdb\test\pos'
file_list_neg_test = glob.glob(path_test_neg + "/*.txt")
file_list_pos_test = glob.glob(path_test_pos + "/*.txt")
test_neg_reviews=[]
for i in range(0,len(file_list_neg_test)):
    data=pd.read_table(file_list_neg_test[i])
    test_neg_reviews.append(data.columns.tolist()[0])
test_pos_reviews=[]
for i in range(0,len(file_list_pos_test)):
    data=pd.read_table(file_list_pos_test[i])
    test_pos_reviews.append(data.columns.tolist()[0])

test_x=test_pos_reviews+test_neg_reviews
test_y=np.append(np.ones((len(test_pos_reviews))), np.zeros((len(test_neg_reviews))))

In [3]:
df_test = pd.DataFrame(list(zip(test_y, test_x)), columns =['test_y', 'test_x'])


In [4]:
from nltk.corpus import stopwords

In [5]:
def text_to_word_list(text):
    """
    Preprocess and convert texts to a list of words 
    
    """
    stopwords_english = stopwords.words('english')

    text = str(text)
    text = text.lower()
    
    # remove html tags
    text = re.sub(r'<[^>]+>', '', text)
    # remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    # remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Single character removal
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    
    text = text.split()
    clean_text=[]
    for word in text:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
              # stemming word
            clean_text.append(word)
        else:
            clean_text.append(" ")
    
    return clean_text

In [6]:
df_test['test_x']=df_test['test_x'].apply(lambda x: text_to_word_list(x))

In [7]:
df_test['test_x']


0        [ , went,  , saw,  , movie, last, night,  ,  ,...
1        [actor, turned, director, bill, paxton, follow...
2        [ , recreational, golfer,  ,  , knowledge,  , ...
3        [ , saw,  , film,  , sneak, preview,  ,  ,  , ...
4        [bill, paxton,  , taken,  , true, story,  ,  ,...
                               ...                        
24995    [ , occasionally, let,  , kids, watch,  , garb...
24996    [ ,  ,  ,  , anymore,  , pretty, much, reality...
24997    [ , basic, genre,  , thriller, intercut,  ,  ,...
24998    [four, things, intrigued,  ,  ,  ,  , film, fi...
24999    [david, bryce, comments, nearby,  , exceptiona...
Name: test_x, Length: 25000, dtype: object

In [8]:
text_test = [row for row in df_test['test_x']]
phrases_test = Phrases(text_test, min_count=1)
bigram_test = Phraser(phrases_test)
sentences_test = bigram_test[text_test]
sentences_test[0]

[' ',
 'went',
 ' ',
 'saw',
 ' ',
 'movie',
 'last_night',
 ' ',
 ' ',
 'coaxed',
 ' ',
 ' ',
 ' ',
 'friends',
 ' ',
 'mine',
 ' ',
 'admit',
 ' ',
 ' ',
 'reluctant',
 ' ',
 'see',
 ' ',
 ' ',
 ' ',
 ' ',
 'knew',
 ' ',
 'ashton_kutcher',
 ' ',
 ' ',
 ' ',
 'able',
 ' ',
 ' ',
 'comedy',
 ' ',
 'wrong',
 'kutcher',
 'played',
 ' ',
 'character',
 ' ',
 'jake_fischer',
 ' ',
 'well',
 ' ',
 'kevin_costner',
 'played',
 'ben_randall',
 ' ',
 ' ',
 'professionalism',
 ' ',
 'sign',
 ' ',
 'good',
 'movie',
 ' ',
 ' ',
 ' ',
 ' ',
 'toy',
 ' ',
 ' ',
 'emotions',
 ' ',
 'one',
 ' ',
 'exactly',
 ' ',
 ' ',
 'entire',
 'theater',
 ' ',
 ' ',
 'sold',
 ' ',
 ' ',
 'overcome',
 ' ',
 'laughter',
 ' ',
 ' ',
 'first_half',
 ' ',
 ' ',
 'movie',
 ' ',
 ' ',
 'moved',
 ' ',
 'tears',
 ' ',
 ' ',
 'second_half',
 ' ',
 'exiting',
 ' ',
 'theater',
 ' ',
 ' ',
 'saw',
 'many',
 'women',
 ' ',
 'tears',
 ' ',
 'many',
 'full_grown',
 'men',
 ' ',
 'well',
 'trying_desperately',
 ' ',
 ' ',
 'let

In [9]:
w2v_model_test = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model_test.build_vocab(sentences_test, progress_per=50000)

print('Time to build vocab for test: {} mins'.format(round((time() - start) / 60, 2)))

Time to build vocab for test: 0.11 mins


In [10]:
start = time()

w2v_model_test.train(sentences_test, total_examples=w2v_model_test.corpus_count, epochs=30, report_delay=1)

print('Time to train the model test: {} mins'.format(round((time() - start) / 60, 2)))



Time to train the model test: 3.39 mins


In [11]:
w2v_model_test.save("word2vec_test.model")

In [12]:
file_export_test = df_test.copy()
file_export_test['test_x'] = file_export_test['test_x'].apply(lambda x: ' '.join(bigram_test[x]))

In [13]:
file_export_test['test_x'][0]

'  went   saw   movie last_night     coaxed       friends   mine   admit     reluctant   see         knew   ashton_kutcher       able     comedy   wrong kutcher played   character   jake_fischer   well   kevin_costner played ben_randall     professionalism   sign   good movie         toy     emotions   one   exactly     entire theater     sold     overcome   laughter     first_half     movie     moved   tears     second_half   exiting   theater     saw many women   tears   many full_grown men   well trying_desperately     let anyone see   crying   movie   great   suggest     go see       judge'

In [14]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [15]:
file_export_test['test_x'].to_csv("test_data.csv", index=False)


In [16]:
word_vectors_test=w2v_model_test.wv

In [17]:
word_vectors_test.vectors


array([[-0.04780941,  0.14622356,  0.14958371, ...,  0.25098464,
        -0.0602538 , -0.0591798 ],
       [ 0.3445924 ,  0.4375354 ,  0.0115928 , ...,  0.5551704 ,
         0.11971668, -0.22544034],
       [ 0.37882426,  0.02022425,  0.20055208, ...,  0.02144994,
         0.22550687, -0.39498693],
       ...,
       [-0.01057147, -0.06345598, -0.04084519, ..., -0.14383759,
        -0.0158386 , -0.02100298],
       [ 0.13612935, -0.0529405 , -0.07582344, ..., -0.1239469 ,
        -0.08291385,  0.05322327],
       [ 0.05790737, -0.11217798,  0.0027252 , ..., -0.04053995,
        -0.04258352, -0.02639282]], dtype=float32)

In [18]:
from sklearn.decomposition import PCA


In [19]:
pca = PCA(n_components = 10)
word_vectors_test.vectors = pca.fit_transform(word_vectors_test.vectors)


In [20]:
word_vectors_test[0]

array([-0.9934438 ,  0.6696675 , -0.81136155, -0.20199941,  0.88867635,
        0.01777962,  0.44718662, -0.1768033 ,  0.5404171 , -0.31571737],
      dtype=float32)

In [21]:
model_test=KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors_test.vectors.astype('double'))

In [22]:
model_test.cluster_centers_

array([[-8.07004111e-01, -3.19967674e-02, -1.89575399e-02,
         2.99364983e-02,  2.10783659e-02,  8.20303736e-04,
         1.17997111e-02, -3.11230764e-04,  1.10617553e-03,
        -1.25227332e-03],
       [ 8.66757626e-01,  3.43647084e-02,  2.03611499e-02,
        -3.21532850e-02, -2.26401349e-02, -8.80605365e-04,
        -1.26738971e-02,  3.33946664e-04, -1.18784607e-03,
         1.34578841e-03]])

In [23]:
word_vectors_test.similar_by_vector(model_test.cluster_centers_[1], topn=100, restrict_vocab=None)


[('unknowingly', 0.9851993322372437),
 ('riccardo', 0.9774933457374573),
 ('gathers', 0.9709517359733582),
 ('yuk_su', 0.9683074355125427),
 ('accidentally_killed', 0.9680965542793274),
 ('momma_boy', 0.9644542932510376),
 ('lederer', 0.9642254710197449),
 ('goo_goo', 0.9638940691947937),
 ('willoughby', 0.9626596570014954),
 ('recently_deceased', 0.9602170586585999),
 ('pinkerton', 0.9592806696891785),
 ('braves', 0.9587650299072266),
 ('trusts', 0.9582815170288086),
 ('retires', 0.9581400752067566),
 ('master_yat', 0.9572418332099915),
 ('illegal_immigrant', 0.9570226073265076),
 ('siu_yu', 0.9566438794136047),
 ('john_eldredge', 0.9562990665435791),
 ('evee', 0.9557904005050659),
 ('rancher', 0.9545687437057495),
 ('former_girlfriend', 0.9543712735176086),
 ('tow', 0.9542824029922485),
 ('evelyn_keyes', 0.9529438614845276),
 ('young_orphan', 0.9506043195724487),
 ('mcbee', 0.9505482316017151),
 ('flees', 0.9503291845321655),
 ('kensuke', 0.9500924348831177),
 ('juanita', 0.949966967

In [24]:
word_vectors_test.most_similar('bad',topn=15)

[('worse', 0.9694814085960388),
 ('sucks', 0.9684250354766846),
 ('horrible', 0.9671058654785156),
 ('terrible', 0.9616751670837402),
 ('even_worse', 0.9559447765350342),
 ('awful', 0.950816810131073),
 ('coherent_script', 0.9444460868835449),
 ('normal_scale', 0.939327597618103),
 ('minutes_tops', 0.9386139512062073),
 ('lame', 0.9349110126495361),
 ('ever_seen', 0.9316216707229614),
 ('even', 0.931445300579071),
 ('product_placement', 0.930504560470581),
 ('tornado_tommy', 0.9297540783882141),
 ('funny_bits', 0.9290056228637695)]

In [25]:
positive_cluster_center = model_test.cluster_centers_[0]
negative_cluster_center = model_test.cluster_centers_[1]

In [26]:
words_test = pd.DataFrame(word_vectors_test.index_to_key)

In [27]:
words_test.columns = ['words']

In [28]:
words_test['vectors'] = words_test.words.apply(lambda x: word_vectors_test[f'{x}'])


In [29]:
len(words_test['vectors'][1])

10

In [30]:
start = time()

words_test["cluster"] = words_test["vectors"].apply(lambda x: model_test.predict([x.astype('double')]))
print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))


Time to train the model: 41.63 mins


In [31]:
words_test

,words,vectors,cluster
0,,"[-0.9934438, 0.6696675, -0.81136155, -0.201999...",[0]
1,movie,"[-2.8646297, 1.4718636, -0.92826766, 0.0476076...",[0]
2,film,"[-2.8255777, -0.80261135, -0.5002815, -0.12243...",[0]
3,one,"[-1.2143981, 1.0122045, -0.4404688, -0.0133457...",[0]
4,like,"[-1.4653059, 1.7439641, -0.30428168, -0.303772...",[0]
...,...,...,...
54646,cha,"[-0.59306616, 0.33769694, -0.09070069, 0.05452...",[0]
54647,danged,"[0.088530414, 0.21273738, 0.2350936, 0.0089622...",[1]
54648,corset,"[0.15470609, 0.17892982, -0.19603989, -0.05230...",[1]
54649,neutralizing,"[-0.008716064, -0.16565132, -0.7028671, -0.176...",[0]


In [32]:
words_test.cluster = words_test.cluster.apply(lambda x: x[0])
words_test['cluster_value'] = [1 if i==0 else -1 for i in words_test.cluster]
words_test['closeness_score'] = words_test.apply(lambda x: 1/(model_test.transform([x.vectors]).min()), axis=1)
words_test['sentiment_coeff'] = words_test.closeness_score * words_test.cluster_value

In [33]:
words_test.head(10)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,,"[-0.9934438, 0.6696675, -0.81136155, -0.201999...",0,1,0.623509,0.623509
1,movie,"[-2.8646297, 1.4718636, -0.92826766, 0.0476076...",0,1,0.283096,0.283096
2,film,"[-2.8255777, -0.80261135, -0.5002815, -0.12243...",0,1,0.378917,0.378917
3,one,"[-1.2143981, 1.0122045, -0.4404688, -0.0133457...",0,1,0.602923,0.602923
4,like,"[-1.4653059, 1.7439641, -0.30428168, -0.303772...",0,1,0.419065,0.419065
5,good,"[-2.0575356, 0.94678575, 0.9828352, -0.4168552...",0,1,0.332057,0.332057
6,would,"[-1.7099445, 1.0980903, -1.3524666, 1.049664, ...",0,1,0.361347,0.361347
7,time,"[-1.327281, 1.0317596, -0.9380007, 0.57386595,...",0,1,0.458329,0.458329
8,really,"[-2.0235353, 1.2417774, -0.46951735, -1.190692...",0,1,0.301810,0.301810
9,see,"[-1.5053174, 1.3558956, -1.078633, 0.7393307, ...",0,1,0.380213,0.380213


In [34]:
words_test[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary_test.csv', index=False)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [36]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [37]:
sentiment_map_test = pd.read_csv("sentiment_dictionary_test.csv")
sentiment_dict_test = dict(zip(sentiment_map_test["words"].values, sentiment_map_test["sentiment_coeff"].values))

In [38]:
test=pd.read_csv("test_data.csv")

In [39]:
tfidf_test = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
tfidf_test.fit(test['test_x'].values.astype('U'))
features_test = pd.Series(tfidf_test.get_feature_names())
review_tfidf_test = tfidf_test.transform(test['test_x'].values.astype('U'))

C:\Users\raja4\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\raja4\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [40]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x['test_x'].split()))

In [41]:
test['test_x']

0          went   saw   movie last_night     coaxed    ...
1        actor turned director bill_paxton follows     ...
2          recreational golfer     knowledge     sport ...
3          saw   film   sneak_preview       delightful ...
4        bill_paxton   taken   true story     us golf o...
                               ...                        
24995      occasionally let   kids watch   garbage     ...
24996            anymore   pretty_much reality tv_shows...
24997      basic genre   thriller intercut     uncomfor...
24998    four things intrigued         film firstly   s...
24999    david bryce comments nearby   exceptionally_we...
Name: test_x, Length: 25000, dtype: object

In [42]:
%%time
replaced_tfidf_scores_test = test.apply(lambda x: replace_tfidf_words(x, review_tfidf_test, features_test), axis=1)

CPU times: total: 10.7 s
Wall time: 11.1 s


In [43]:
def replace_sentiment_words(word, sentiment_dict):
    """
    replacing each word with its associated sentiment score from sentiment dict
    """
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [44]:
replaced_closeness_scores_test = test['test_x'].apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict_test), str(x).split())))


In [45]:
replacement_df_test = pd.DataFrame(data=[replaced_closeness_scores_test, replaced_tfidf_scores_test, test['test_x']]).T
replacement_df_test.columns = ["sentiment_coeff", "tfidf_scores", "review"]
replacement_df_test["sentiment_rate"] = replacement_df_test.apply(lambda x: np.array(x.loc["sentiment_coeff"]) @ np.array(x.loc["tfidf_scores"]), axis=1)
replacement_df_test['prediction'] = (replacement_df_test.sentiment_rate>0).astype('int8')



In [46]:
replacement_df_test

,sentiment_coeff,tfidf_scores,review,sentiment_rate,prediction
0,"[0.2437598677548857, 0.1938759915768454, 0.283...","[3.9337368818345597, 6.4654519296287996, 5.986...",went saw movie last_night coaxed ...,40.662468,1
1,"[0.191906983386557, 0.4077534436707549, 0.2678...","[3.69418729513324, 4.379084576221043, 3.104102...",actor turned director bill_paxton follows ...,115.988909,1
2,"[-0.9953220646976476, -0.6556286576736207, 0.2...","[9.740376741930469, 9.334911633822305, 5.65861...",recreational golfer knowledge sport ...,54.069739,1
3,"[0.1938759915768454, 0.3789167259852144, 0.251...","[3.2327259648143998, 4.810478216223833, 7.9077...",saw film sneak_preview delightful ...,74.919774,1
4,"[0.2942555339698585, -0.4046634064908467, 0.31...","[7.660935200250632, 4.360479388390009, 3.65530...",bill_paxton taken true story us golf o...,99.515839,1
...,...,...,...,...,...
24995,"[0.2467135571075348, 0.3197506561352115, -0.16...","[5.589336836031823, 3.6289094024277904, 4.0233...",occasionally let kids watch garbage ...,35.008194,1
24996,"[0.240237924442717, 0.2989973164576753, 0.2505...","[5.3245527276732965, 4.910065001965494, 4.4370...",anymore pretty_much reality tv_shows...,106.203477,1
24997,"[0.2116394030408076, 0.1953313356150229, 0.180...","[5.554517070872595, 4.339954152445278, 9.56558...",basic genre thriller intercut uncomfor...,182.540938,1
24998,"[0.3013105718953794, 0.2999432151159756, 0.238...","[4.746548566150594, 3.1621677394988112, 6.5940...",four things intrigued film firstly s...,75.247665,1


In [47]:
predicted_classes = replacement_df_test.prediction
y_test = test_y

conf_matrix = pd.DataFrame(confusion_matrix(df_test['test_y'], replacement_df_test.prediction))
print('Confusion Matrix')
display(conf_matrix)

test_scores = accuracy_score(y_test,predicted_classes), precision_score(y_test, predicted_classes), recall_score(y_test, predicted_classes), f1_score(y_test, predicted_classes)

print('\n \n Scores')
scores = pd.DataFrame(data=[test_scores])
scores.columns = ['accuracy', 'precision', 'recall', 'f1']
scores = scores.T
scores.columns = ['scores']
display(scores)

Confusion Matrix


,0,1
0,1828,10672
1,2332,10168



 
 Scores


,scores
accuracy,0.479840
precision,0.487908
recall,0.813440
f1,0.609958
